In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from loader import get_dataset
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [8]:
codes = np.load("/Users/jerms/Documents/work/mir/datasets/codes/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--3.wav-39.npy", allow_pickle=True)

In [17]:
np.pad(codes[0], (0, ), mode="constant").shape

(1410,)

In [33]:
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

AUTOTUNE = tf.data.experimental.AUTOTUNE

def read_codes(item):
    codes = np.load(item.decode(), allow_pickle=True)
    codes = codes[1].reshape(256, 22, 1)
    return codes.astype(np.float32)

def load_file(filepath):
    codes = tf.numpy_function(read_codes, [filepath], [tf.float32])
    return codes

def split_data(ds, shuffle_buffer_size=1024, batch_size=64):
    test_ds = ds.take(200) 
    train_ds = ds.skip(200)
        
    train_ds = train_ds.shuffle(buffer_size=shuffle_buffer_size)
    train_ds = train_ds.map(load_file, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.batch(batch_size, drop_remainder=True)
    
    test_ds = test_ds.shuffle(buffer_size=shuffle_buffer_size)
    test_ds = test_ds.map(load_file, num_parallel_calls=AUTOTUNE)
    test_ds = test_ds.batch(batch_size, drop_remainder=True)
   
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds.prefetch(tf.data.AUTOTUNE)
    
    return train_ds, test_ds

dataset = get_dataset(ds_dir="/Users/jerms/Documents/work/mir/datasets/codes")
train_ds, test_ds = split_data(dataset)

top_input_size, bot_input_size = (128, 12, 1), (256, 22, 1)

# Define a Pixel CNN network
dist = tfd.PixelCNN(
    image_shape=bot_input_size,
    num_resnet=1,
    num_hierarchies=2,
    num_filters=32,
    num_logistic_mix=5,
    dropout_p=.3,
)

# Define the model input
image_input = tfkl.Input(shape=bot_input_size)

# Define the log likelihood for the loss fn
log_prob = dist.log_prob(image_input)

# Define the model
model = tfk.Model(inputs=image_input, outputs=log_prob)
model.add_loss(-tf.reduce_mean(log_prob))

# Compile and train the model
model.compile(
    optimizer=tfk.optimizers.Adam(.001),
    metrics=[])


In [29]:
for sample in train_ds.take(1):
    print(sample)

(<tf.Tensor: shape=(64, 256, 22, 1), dtype=float32, numpy=
array([[[[414.],
         [466.],
         [466.],
         ...,
         [466.],
         [249.],
         [249.]],

        [[ 14.],
         [198.],
         [105.],
         ...,
         [ 54.],
         [413.],
         [113.]],

        [[ 14.],
         [419.],
         [419.],
         ...,
         [453.],
         [ 42.],
         [453.]],

        ...,

        [[ 54.],
         [331.],
         [413.],
         ...,
         [413.],
         [331.],
         [331.]],

        [[ 54.],
         [331.],
         [331.],
         ...,
         [413.],
         [331.],
         [331.]],

        [[ 54.],
         [331.],
         [331.],
         ...,
         [331.],
         [331.],
         [331.]]],


       [[[414.],
         [414.],
         [466.],
         ...,
         [466.],
         [249.],
         [249.]],

        [[ 14.],
         [198.],
         [453.],
         ...,
         [198.],
         [113.],


In [31]:
dataset = dataset.map(load_file, num_parallel_calls=AUTOTUNE)

In [ ]:
model.fit(train_ds, epochs=10, verbose=True)

# sample five images from the trained model
samples = dist.sample(5)

Epoch 1/10
   7/5491 [..............................] - ETA: 13:17:00 - loss: nan